## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
1,3,Atuona,PF,77.07,light rain,-9.8000,-139.0333,Villa Enata
2,4,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park
3,7,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,People ThankYou
4,11,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
1,3,Atuona,PF,77.07,light rain,-9.8000,-139.0333,Villa Enata
2,4,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park
3,7,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,People ThankYou
4,11,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()
preferred_cities_df.count()

City_ID                209
City                   209
Country                209
Max Temp               209
Current Description    209
Lat                    209
Lng                    209
Hotel Name             209
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
1,3,Atuona,PF,77.07,light rain,-9.8000,-139.0333,Villa Enata
2,4,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park
3,7,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,People ThankYou
4,11,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique
...,...,...,...,...,...,...,...,...
204,690,Talara,PE,69.64,few clouds,-4.5772,-81.2719,Hostal Oro Negro
205,692,Nanma,CN,66.13,light rain,36.1817,118.1583,GreenTree Inn Zibo Wuyuan County Lushan Road E...
206,693,Noumea,NC,66.18,scattered clouds,-22.2763,166.4572,Hôtel Le Lagon
207,694,Pochutla,MX,75.81,scattered clouds,15.7432,-96.4661,Hotel Posada San Jose


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,
1,Atuona,PF,77.07,light rain,-9.8000,-139.0333,
2,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,
3,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,
4,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,
5,Shiyan,CN,71.40,light rain,32.6475,110.7781,
6,Saint-Philippe,RE,74.79,scattered clouds,-21.3585,55.7679,
7,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,
8,Cape Town,ZA,69.35,clear sky,-33.9258,18.4232,
9,Tukrah,LY,79.99,few clouds,32.5341,20.5791,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Cannot find hotel...skipping.")

In [12]:
hotel_df.count()

City                   218
Country                218
Max Temp               218
Current Description    218
Lat                    218
Lng                    218
Hotel Name             218
dtype: int64

In [13]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
3,Atuona,PF,77.07,light rain,-9.8000,-139.0333,Villa Enata
4,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park
7,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,People ThankYou
11,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique


In [9]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Bambous Virieux,MU,74.21,broken clouds,-20.3428,57.7575,Casa Tia Villa
1,Atuona,PF,77.07,light rain,-9.8000,-139.0333,Villa Enata
2,Mareeba,AU,68.72,broken clouds,-17.0000,145.4333,Trinity Plains Tourist Park
3,Rikitea,PF,72.03,light rain,-23.1203,-134.9692,People ThankYou
4,La Libertad,EC,70.45,overcast clouds,-2.2333,-80.9000,Playa Canela Hotel Boutique
...,...,...,...,...,...,...,...
204,Talara,PE,69.64,few clouds,-4.5772,-81.2719,Hostal Oro Negro
205,Nanma,CN,66.13,light rain,36.1817,118.1583,GreenTree Inn Zibo Wuyuan County Lushan Road E...
206,Noumea,NC,66.18,scattered clouds,-22.2763,166.4572,Hôtel Le Lagon
207,Pochutla,MX,75.81,scattered clouds,15.7432,-96.4661,Hotel Posada San Jose


In [10]:
# 8a. Create the output File (CSV)
output_data_file=("Weather_Database/WeatherPy_Database.csv")
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

##### 